# Boundary Guided Adversarial Training (BGAT)

In [1]:
import os
import sys
sys.path.append('..')
import yaml
import shutil
import argparse
import numpy as np
from tqdm import tqdm
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, random_split

from utils import *

## Parameter setting

In [2]:
gpu = '4,5,6,7'
dataset = 'cifar10'
model_type = 'wrn34-10'
checkpoint = './checkpoint/bgat/%s/%s' % (model_type, dataset)
num_classes = 10
lr = 0.1
momentum = 0.9
weight_decay = 2e-4
batch_size = 128
total_epochs = 100
lam = 1
epsilon = 0.031
alpha = 0.007
num_repeats = 10

## Inner maximization

In [3]:
def inner_max(model, xent, inputs, targets, epsilon, alpha, num_repeats):
    noise = torch.FloatTensor(inputs.shape).uniform_(-epsilon, epsilon).cuda()
    x = torch.clamp(inputs + noise, min=0, max=1)
    
    for _ in range(num_repeats):
        x.requires_grad_()
        logits = model(x)
        loss = xent(logits, targets)
        loss.backward()
        grads = x.grad.data
        x = x.detach() + alpha*torch.sign(grads).detach()
        x = torch.min(torch.max(x, inputs-epsilon), inputs+epsilon).clamp(min=0, max=1)
    return x

## Training (Outer minimization)

In [4]:
def standard_training(epoch, model, dataloader, optimizer):
    model.train()
    total = 0
    total_loss = 0
    total_correct = 0
    xent = nn.CrossEntropyLoss()
    
    for idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        batch = inputs.size(0)
        logits = model(inputs)
        loss = xent(logits, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total += batch
        total_loss += loss.item()
        num_correct = logits.softmax(dim=1).argmax(dim=1).eq(targets).sum().item()
        total_correct += num_correct
        
        if idx % 100 == 0:
            print('Epoch %d [%d/%d] | loss: %.4f (avg: %.4f) | acc: %.4f (avg: %.4f) |'\
                  % (epoch, idx, len(dataloader), loss.item(), 
                     total_loss/len(dataloader), 
                     num_correct/batch, total_correct/total))

In [5]:
def training(epoch, model_nat, model_rob, dataloader, optimizer, num_classes, 
             lam=1, epsilon=8/255, alpha=2/255, num_repeats=10):
    model_nat.eval()
    model_rob.train()
    total = 0
    total_loss = 0
    total_correct = 0
        
    mse = nn.MSELoss()
    xent = nn.CrossEntropyLoss()
    for idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        batch = inputs.size(0)
        
        with torch.no_grad():
            logits_nat = model_nat(inputs)
        x = inner_max(model_rob, xent, inputs, targets, epsilon, alpha, num_repeats)
        logits_rob = model_rob(x)
        
        loss = mse(logits_rob, logits_nat)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        total += batch
        total_loss += loss.item()
        num_correct = torch.argmax(logits_rob.data, dim=1).eq(targets.data).cpu().sum()
        total_correct += num_correct
        
        if idx % 100 == 0:
            print('Epoch %d [%d/%d] | loss: %.4f (avg: %.4f) | acc nat: %.4f (avg: %.4f) |'\
                  % (epoch, idx, len(dataloader), loss.item(), 
                     total_loss/len(dataloader), 
                     num_correct/batch, total_correct/total))

In [6]:
def evaluation(epoch, model, dataloader, alpha, epsilon, num_repeats, use_robust=False):
    model.eval()
    total_correct_nat = 0
    total_correct_adv = 0
    
    xent = nn.CrossEntropyLoss()
    with tqdm(dataloader) as pbar:
        for index, (inputs, targets) in enumerate(pbar):
            inputs, targets = inputs.cuda(), targets.cuda()
            batch = inputs.size(0)
            pbar.set_description("[eval] %d/%d" % (index+1, len(dataloader.dataset)))
            if use_robust:
                with torch.enable_grad():
                    x = inner_max(model, xent, inputs, targets, epsilon, alpha, num_repeats)
            
                with torch.no_grad():
                    logits_nat = model(inputs)
                    logits_adv = model(x)
                total_correct_nat += torch.argmax(logits_nat.data, dim=1).eq(targets.data).cpu().sum().item()
                total_correct_adv += torch.argmax(logits_adv.data, dim=1).eq(targets.data).cpu().sum().item()
                pbar.set_postfix(OrderedDict(acc_nat=total_correct_nat / len(dataloader.dataset),
                                             acc_rob=total_correct_adv / len(dataloader.dataset)))
            else:
                with torch.no_grad():
                    logits_nat = model(inputs)
        
                total_correct_nat += torch.argmax(logits_nat.data, dim=1).eq(targets.data).cpu().sum().item()
                pbar.set_postfix(OrderedDict(acc_nat=total_correct_nat / len(dataloader.dataset)))
    
        return (total_correct_nat / len(dataloader.dataset)), (total_correct_adv / len(dataloader.dataset))


In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = gpu
os.makedirs(checkpoint, exist_ok=True)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()])
train_dataset, _ = get_dataloader(dataset, batch_size)
num_samples = len(train_dataset)
num_samples_for_train = int(num_samples * 0.98)
num_samples_for_valid = num_samples - num_samples_for_train
train_set, valid_set = random_split(train_dataset, [num_samples_for_train, num_samples_for_valid])
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=False)
valid_dataloader = DataLoader(valid_set, batch_size=1, shuffle=True, drop_last=False)

model = nn.DataParallel(get_network(model_type, num_classes).cuda())
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

scheduler = [75, 90]
adjust_learning_rate = lr_scheduler.MultiStepLR(optimizer, scheduler, gamma=0.1)
best_acc = 0
# Training natural model
for epoch in range(total_epochs):
    standard_training(epoch, model, train_dataloader, optimizer)
    test_acc_nat, _ = evaluation(epoch, model, valid_dataloader, alpha, epsilon, num_repeats, use_robust=False)
    is_best = best_acc < test_acc_nat
    best_acc = max(best_acc, test_acc_nat)
    save_checkpoint = {'state_dict': model.state_dict(),
                       'best_acc': best_acc,
                       'optimizer': optimizer.state_dict(),
                       'model_type': model_type,
                       'dataset': dataset}
    torch.save(save_checkpoint, os.path.join(checkpoint, 'model_nat'))
    if is_best:
        torch.save(save_checkpoint, os.path.join(checkpoint, 'best_model_nat'))
    adjust_learning_rate.step()
    
print('Start BGAT.')
model_nat = nn.DataParallel(get_network(model_type, num_classes).cuda())
state_dict_nat = torch.load(os.path.join(checkpoint, 'best_model_nat'))['state_dict']
model_nat.load_state_dict(state_dict_nat)
model_rob = nn.DataParallel(get_network(model_type, num_classes).cuda())
optimizer_rob = optim.SGD(model_rob.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
adjust_learning_rate = lr_scheduler.MultiStepLR(optimizer_rob, scheduler, gamma=0.1)

best_acc_nat = 0
best_acc_rob = 0
# Training robust model using natural model
for epoch in range(total_epochs):
    training(epoch, model_nat, model_rob, train_dataloader, optimizer_rob, num_classes, lam, epsilon, alpha, num_repeats)
    test_acc_nat, test_acc_rob = evaluation(epoch, model_rob, valid_dataloader, alpha, epsilon, num_repeats, use_robust=True)
    is_best = best_acc_nat < test_acc_nat and best_acc_rob < test_acc_rob
    best_acc_nat = max(best_acc_nat, test_acc_nat)
    best_acc_rob = max(best_acc_rob, test_acc_rob)
    save_checkpoint = {'state_dict_rob': model_rob.state_dict(),
                       'best_acc_rob': best_acc_rob,
                       'optimizer': optimizer_rob.state_dict(),
                       'model_type': model_type,
                       'dataset': dataset}
    torch.save(save_checkpoint, os.path.join(checkpoint, 'model_rob'))
    if is_best:
        torch.save(save_checkpoint, os.path.join(checkpoint, 'best_model_rob'))
    adjust_learning_rate.step()

Files already downloaded and verified
Files already downloaded and verified
Epoch 0 [0/383] | loss: 2.3289 (avg: 0.0061) | acc: 0.0859 (avg: 0.0859) |
Epoch 0 [100/383] | loss: 1.7107 (avg: 0.5210) | acc: 0.3125 (avg: 0.2710) |
Epoch 0 [200/383] | loss: 1.6129 (avg: 0.9442) | acc: 0.4062 (avg: 0.3336) |
Epoch 0 [300/383] | loss: 1.3646 (avg: 1.3225) | acc: 0.5078 (avg: 0.3776) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [00:20<00:00, 47.67it/s, acc_nat=0.412]


Epoch 1 [0/383] | loss: 1.3437 (avg: 0.0035) | acc: 0.5156 (avg: 0.5156) |
Epoch 1 [100/383] | loss: 1.0840 (avg: 0.3121) | acc: 0.5781 (avg: 0.5723) |
Epoch 1 [200/383] | loss: 1.0257 (avg: 0.5955) | acc: 0.6641 (avg: 0.5923) |
Epoch 1 [300/383] | loss: 1.2346 (avg: 0.8613) | acc: 0.5703 (avg: 0.6077) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [00:20<00:00, 47.81it/s, acc_nat=0.64]


Epoch 2 [0/383] | loss: 0.9155 (avg: 0.0024) | acc: 0.6562 (avg: 0.6562) |
Epoch 2 [100/383] | loss: 0.7442 (avg: 0.2353) | acc: 0.7031 (avg: 0.6863) |
Epoch 2 [200/383] | loss: 0.9118 (avg: 0.4479) | acc: 0.6484 (avg: 0.7017) |
Epoch 2 [300/383] | loss: 0.7649 (avg: 0.6489) | acc: 0.7422 (avg: 0.7114) |


...

[eval] 1000/1000: 100%|██████████| 1000/1000 [04:37<00:00,  3.60it/s, acc_nat=0.876, acc_rob=0.531]


Epoch 98 [0/383] | loss: 1.5365 (avg: 0.0040) | acc nat: 0.9062 (avg: 0.9062) |
Epoch 98 [100/383] | loss: 1.9924 (avg: 0.4812) | acc nat: 0.8828 (avg: 0.8834) |
Epoch 98 [200/383] | loss: 1.8344 (avg: 0.9532) | acc nat: 0.8906 (avg: 0.8837) |
Epoch 98 [300/383] | loss: 1.8751 (avg: 1.4320) | acc nat: 0.8984 (avg: 0.8833) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [04:35<00:00,  3.63it/s, acc_nat=0.888, acc_rob=0.512]


Epoch 99 [0/383] | loss: 2.1739 (avg: 0.0057) | acc nat: 0.8594 (avg: 0.8594) |
Epoch 99 [100/383] | loss: 1.8550 (avg: 0.4715) | acc nat: 0.8828 (avg: 0.8876) |
Epoch 99 [200/383] | loss: 1.7761 (avg: 0.9473) | acc nat: 0.8750 (avg: 0.8858) |
Epoch 99 [300/383] | loss: 1.4049 (avg: 1.4237) | acc nat: 0.9219 (avg: 0.8846) |


[eval] 900/1000:  90%|████████▉ | 899/1000 [04:09<00:27,  3.64it/s, acc_nat=0.779, acc_rob=0.469]